# Section 1.1: A deep(er) dive into joining and cleaning datasets

As a general rule, data scientists assume that about 80 percent of the time and effort devoted to a project will be spent preparing data for anlaysis. Because most real-world data is split among multiple existing datasets, this invariably means cleaning and joining diffent datasets together in the way that a data scientist needs for analysis. Thus, mastering these skills is essential for undertaking data science.

This section makes extensive use of pandas, the principal Python library for data handling and manipulation. Note that this section assumes some familiarity with pandas and basic Python skills.

In order to provide an experience more like real-world data science, we will use real data taken gathered from the [U.S. Department of Agriculture National Nutrient Database for Standard Reference](https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/nutrient-data-laboratory/docs/usda-national-nutrient-database-for-standard-reference/).

## Reminders about importing, built-in Help, and documentation

The standard convention in Python-centric data science is to import pandas under the alias `pd`, which is what we will use here:

In [ ]:
import pandas as pd

Because this is such a common convention, it is the way we will use and refer to pandas throughout the rest of this section and this course. You should also adopt this usage in your own code to make it easily readable for other data scientists.

Pandas is a big package and there can be a lot to keep track of. Fortunately, IPython (the underlying program that powers this notebook and other like it) gives you the ability to quickly explore the contents of a package like pandas by using its tab-completion feature. If you want to see all of the functions available with pandas, type this:

```ipython
In [2]: pd.<TAB>

```

When you do so, a drop-down menu will appear next to the `pd`.

> **Exercise**

In [ ]:
# Place your cursor after the period and press <TAB>:
pd.

# Now select an item using tab-completion and then add a period
# and use tab-completion to explore *that*.
# For example, you could try placing pressing <TAB> after:
# pd.DataFrame.


As we progress through Section 1.1, don't forget that IPython also provides a tab-completion feature and function documentation with the `?` (question mark) character. If you don't understand something about a function you see in this section, taking a moment to csonult the documentation can help a great deal. You will find this documentation to be a very valuable reference source for your own data science work, both now and in the future. As a reminder, use this code to display the built-in pandas documentation:

```ipython
In [4]: pd?
```

> **Exercise**

> Run this code cell and review the documentation for the pandas DataFrame object. We are going to use it quite a bit.

In [ ]:
pd.DataFrame.?

## A brief reminder about Jupyter notebooks

This course makes extensive use of Jupyter notebooks hosted on Microsoft Azure. Azure-hosted Jupyter notebooks provide an easy way for you to experiment with programming concepts in an interactive fashion that requires no installation of software by students on local computers.

Jupyter notebooks are divided into cells. Each cell either contains text written in the Markdown markup language or a space in which to write and execute computer code. Because all the code resides inside code cells, you can run each code cell inline rather than using a separate Python interactive window.

> **Note**: This notebook is designed to have you run code cells one by one, and several code cells contain deliberate errors for demonstration purposes. As a result, if you use the **Cell** > **Run All** command, some code cells past the error won't be run. To resume running the code in each case, use **Cell** > **Run All Below** from the cell after the error.

## Loading data

> **Learning goal:** By the end of this subsection, you should be comfortable loading data from files into panda `DataFrame`s and troubleshooting any difficulties that might arise.

Because pandas `DataFrame`s are two-dimensional data structures, they are inherently similar to flat-file formats such as comma-separated value (CSV) files, the most common import and export format for spreadsheets and databases. Adding to this ease of translation from CSV files to `DataFrame`s, pandas provides a convenient function to load the contents of CSV files into `DataFrame`s (more convenient, in fact, then the native Python [CSV library](https://docs.python.org/3.6/library/csv.html)). Let's get comfortable with [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) because we will be using often.

In [ ]:
df = pd.read_csv('Data/USDA-nndb.csv')

After talking up the convenience of pd.read_csv, it might seem strange that we immediately encounter an error. The clue as to what went wrong is in the last line of the error message:

`UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte`

The data in the CSV file uses a character that the default Unicode Standard ([UTF-8](https://en.wikipedia.org/wiki/UTF-8)) codec reading this file doesn't understand. Remember, this is real-world data and the real world is a messy place.

It's time to use the pd.read_csv documentation to look for ideas on what to try next.

> **Exercise**

> Use the built-in IPython documentation to on `pd.read_csv.`

In [ ]:
# Hint: Refer to the discussion at the start of this section if
# you forgot the syntax.


There are quite a few of parameters for this function. The intuitively named `encoding` parameter accepts `str` input from the list of Python [standard encodings](https://docs.python.org/3.6/library/codecs.html#standard-encodings). We will go with `'latin_1'` here.

**Note:** Although data-science practitioners do develop a familiarity with different encodings they have encountered, selecting the correct encoding can sometimes come down to trial and error, even for professionals!

In [ ]:
df = pd.read_csv('Data/USDA-nndb.csv', encoding='latin_1')

There was no error message this time, so `'latin_1'` did the trick and we successfully read in the CSV file to the `df` `DataFrame`.

> **Takeaway:** There is a saying that the difference between data science in academia and the real world is that academia likes to do complex analysis on clean datasets, whereas the real world often does simpler analysis on messier datasets. Troubleshooting difficulties — even ones encountered while merely loading your data — is a large part of successful data science.

## Exploring and cleaning the data

> **Learning goal:** By the end of this subsection, you should be comfortable performing simple exploration of your data and performing simple cleaning steps on it to prepare it for later analysis.

Let's now take a look at our `DataFrame`:

In [ ]:
df.head()

We can get some more specific information about the `DataFrame` by using its `info()` method:

In [ ]:
df.info()

Just quickly inspecting the columns from `df` we can see that almost all of the columns have a number of null values. Those missing values are not an issue for us right now, but they will pose a challenge in future sections (but we will deal with them in those sections).

Let's also check to see if this `DataFrame` has any duplicate values in it. We'll do this by looking at the `NDB_No` column, which serves as the unique identifier for all foods in the National Nutrient Database for Standard Reference.

In [ ]:
df.duplicated('NDB_No').sum()

Given the nature of the data source (a government reference database) it makes sense that there are no duplicate entries. For purposes of learning more about cleaning data, let's duplicate data by using the `append()` method.

In [ ]:
df = df.append(df, ignore_index=True)

The `append()` method has basically stacked the `DataFrame` by appending a copy of `df` to the end of the `DataFrame`. The `ignore_index=True` parameter means that the index numbering for the newly doubled `DataFrame` continues seamlessly.

Now let's look directly at how many times individual values in a column (such as `NDB_No`) are duplicated using the `pivot_table` method for the `df` `DataFrame`:

In [ ]:
dups = df.pivot_table(index=['NDB_No'], aggfunc='size')
print(dups)

> **Exercise**
>
> You can create a spreadsheet-style pivot table on this DataFrame by using more than one column. Can you figure out how to do this for the `NDB_No` and `Shrt_Desc` columns?

In [ ]:
# Hint: refer to the documentation using df.pivot_table? if you need help.


Given that we duplicated the original dataset, two duplicates of everything is not unexpected. However, these duplicate values will pose a problem for us later in the section if not dealt with, so let's take care of them now:

In [ ]:
df = df.drop_duplicates('NDB_No', keep="last")
df.info()

The `DataFrame` is now half of its original size, which is what we would expect. However, look at this line in the `df.info()` output:

`Int64Index: 8790 entries, 8790 to 17579`

While there are only now 8790 entries per column, the indexing for the DataFrame does not run 0 through 8789, as we might have expected. We can see this more directly by looking at the `head` of the redacted `DataFrame`:

In [ ]:
df.head()

> **Question**
>
> Is this behavior of the `drop_duplicates()` method not updating the index values of the `DataFrame` surprising or unexpected for you? Can you explain why this method behaves as it does in this case? If not, study the documentation for this method by using `df.drop_duplicates?` in the code cell below until you're satisfied with your understanding of this behavior.

> **Takeaway:** Duplicate, `Null`, and `NaN` values can all complicate (if not derail) your analysis. Learning how to identify and remove these problems is a huge part of successfully performing data science.

## Splitting the `DataFrame`

> **Learning goal:** By the end of this subsection, you should be comfortable selecting and dropping specific columns from a `DataFrame`.

It might seem strange to discuss splitting a `DataFrame` in a course section on joining them, but we'll do so here to create the `DataFrame`s that we'll join later on. We take this approach for two reasons:

1. Creating our own `DataFrame`s gives us easy control over the content of the child `DataFrame`s to best demonstrate aspects of joining datasets.
2. Because we have a baseline, joined `DataFrame` (`df`), it's easy to see how different methods of joining the child `DataFrame`s produce different results.

We'll create two child `DataFrame`s from `df`: `df1` and `df2`. First, we'll create `df1` from the first 35 columns of `df`.

In [ ]:
df1 = df.iloc[:,:35]

> **Exercise**

> Why did we use syntax `df1 = df.iloc[:,:35]` to capture the first 65 columns of `df`? What does the first `:` (colon) in the square brackets do? Experiment with `df3 = df.iloc[:35]` in the code cell below and compare `df3.info()` with `df1.info()` to satisfy yourself as to why we need to use this syntax.

In [ ]:
df1.info()

We'll create `df2` in a similar manner to `df1`, but we need to do things a little differently here to ensure that the first columne (`NDB_No`) makes it into `df2`. This is going to serve as the column that's common to both child `DataFrame`s when we join them later in this section.

We also want to populate `df2` with a different number of rows than `df1`. Doing so will make is easier to demonstrate what goes on with some of the join techniques shown below.

In [ ]:
df2 = df.iloc[0:2000, [0]+[i for i in range(35,53)]]

> **Question**

> If you're unsure about why we use `range(35,53)` in the list comprehension above, review the documentation for the `range()` function using `range?` in the code cell below.

In [ ]:
# Remeber that Python uses zero-indexing.


We can examine `df2` by using the `head()` and `info()` methods.

In [ ]:
df2.head()

In [ ]:
df2.info()

Let’s take a look at `df1`.

In [ ]:
df1.head()

You'll notice on that both `DataFrame`s have their old indices indexes that they inherited from  `df`. We can fix that by using the `reset_index()` method, but then we run into a problem.

In [ ]:
df1 = df1.reset_index()
df1.head()

Our old indexes are still there for `df1`, but now they're in a new column titled "`index`." We need to address that by using the `drop=True` parameter for the method. (We also need to drop the `index` column we just created.)

In [ ]:
df1 = df1.drop(['index'], axis=1)
df1 = df1.reset_index(drop=True)
df1.head()

Now let's do the same thing to `df2`.

In [ ]:
df2 = df2.reset_index(drop=True)
df2.head()

For practice, let's export these `DataFrame`s to CSV files by using the `to_csv()` method. Note that unless we explicitly tell pandas not to, it will also export the index as a column in the CSV file. We will also need to be careful to explicitly encode our CSV to UTF-8.

In [ ]:
df1.to_csv('Data/NNDB1.csv', sep=',', encoding='utf-8',index=False)

> **Exercise**

In [ ]:
# Export df2 to a CSV file.


> **Takeaway:** Although it's not common in the real world to split `DataFrame`s only to re-merge them later, you'll need to drop columns or create new `DataFrame`s that contain only the information you need. With truly large datasets, this is not just a convenience for you analysis, but a necessity for memory and performance!

## Joining `DataFrame`s

> **Learning goal:** By the end of this subsection, you should be comfortable performing left, right, inner, and outer merges on `DataFrame`s.

We'll examine the  most commonly used `DataFrame` function for joining datasets: `merge()`. But first, let's refresh ourselves on the shapes of our two `DataFrame`s so that the output of our joining makes more sense.

In [ ]:
df1.shape

In [ ]:
df2.shape

The type of dataset join that’s most widely used by practicing data scientists is the left join. If you already have some experience with SQL, you know what this refers to. Basically, a left join is a join that takes all of the data from one `DataFrame` (think of it as the left set in a Venn diagram) and merges it with everything that it has in common with another `DataFrame` (the intersection with the right set in the same Venn diagram).

We do this using the `merge()` function. We also need to specify the type of join we want to perform by using the `how` parameter, as well as the index on which to join the `DataFrames` by using the `on` parameter.

In [ ]:
left_df = pd.merge(df1, df2, how='left', on='NDB_No')
left_df.shape

> **Question**

> Is the shape of the resulting `DataFrame` what you were expecting? Why or why not?

Now let's compare this to the original `df` `DataFrame`.

In [ ]:
df.shape

> **Question**

> The shapes are the same, but do you expect `df` and `left_df` to be identical? If so, why? If not, what differences do you expect there to be between them?

Let's check to see what the differences between these `DataFrame`s might be.

In [ ]:
df.head()

In [ ]:
left_df.head()

The indexes notwithstanding, the first five rows of both `DataFrame`s are the same. Let's check the last five rows.

In [ ]:
df.tail()

In [ ]:
left_df.tail()

There are differences here in the last five rows. Notice that the right-most columns of `left_df` contain have Not a Number (`NaN`) values. This is because the left `DataFrame` was larger than the right `DataFrame`.

> **Exercise**

> A right join is simply the mirror image of a left join in which those entries from the left `DataFrame` that are common with the right `DataFrame` are merged with the right `DataFrame`.
>
>Perform a right join of `df1` and `df2` in the code cell below. But before you do that, ask yourself what shape you expect the resulting `DataFrame` to have? Do you expect it to have any `NaN` values?

In [ ]:
# Hint: the parameter for the right join is how='right'


Another intuitive and widely used type of join is the inner join. This join simply merges entries that are common to both `DataFrame`s, resulting in a `DataFrame` that has no `NaN` values.

In [ ]:
inner_df = pd.merge(df1, df2, how='inner', on='NDB_No')

> **Question**

> Before we examine the shape of the resulting `DataFrame`, what do you predict it will be? Why?

In [ ]:
inner_df.shape

Did `inner_df` behave as you expected it would? Let's briefly examine it by using the `head()` and `tail()` methods.

In [ ]:
inner_df.head()

In [ ]:
inner_df.tail()

The resulting `DataFrame` is essentially the first 2000 rows of the original `df` `DataFrame`.

> **Exercise**

> An outer join is the union of two `DataFrame`s; anything that is in either `DataFrame` with be in the resultant one. Perform an outer join of `df1` and `df2`. What shape do you expect the resulting `DataFrame` to have? How does it differ from the right join of `df1` and `df2`? What differences would there have to be in the shape or content of either `DataFrame` for the outer join of the two to be different from their right join?

## Preparing for coming sections

We will be using the USDA NNDB dataset in Sections 1.2 and 1.3. However, particularly in Section 1.2, we want to include food group information to go with the food entries to aid with interpreting the result of our data analysis in that section. You will add food group information to this USDA dataset in preparation for these coming sections.

First, let's reload our original NNDB dataset so that we have a clean copy.

In [ ]:
df = pd.read_csv('Data/USDA-nndb.csv', encoding='latin_1')

Now let's load in the columns that we want from the older NNDB dataset that includes food groups.

In [ ]:
fg_df = pd.read_csv('Data/USDA-nndb-old.csv', usecols=[' ID', 'FoodGroup'])
fg_df = fg_df.rename(index=str, columns={' ID': 'NDB_No'})
fg_df.head()

Note that `fg_df` does not have the same number of rows as `df`:

In [ ]:
fg_df.shape

> **Exercise**
>
> We need to combine `df` and `fg_df` using the pandas `merge()` function. As you prepare to do so, keep the following considerations front of mind:
> 1. Which type of join should you use to capture all of the information in both datasets? (**Hint:** Look at the `head` and `tail` of the resulting `DataFrame` for clues.)
> 2. In order to put the `FoodGroup` column immediately after the `NDB_No` column, in what order should you enter the two `DataFrame`s into the `merge()` function? (You might need to experiment a couple of times to get the desired order.)
>
> Perform the command to join the `df` and `fg_df` in the code cell below.

> **Exercise solution**
>
> The correct code to use for this is `combined_df = pd.merge(fg_df, df, how='outer', on='NDB_No')`.

In [ ]:
combined_df.head()

In [ ]:
combined_df.tail()

Now save the merged `DataFrame` using the `to_csv()` method.

In [ ]:
combined_df.to_csv('Data/USDA-nndb-merged.csv', 
                   sep=',', 
                   encoding='latin_1', 
                   index=False)

> **Takeaway:** Because the most interesting insights come from joining different datasets, the pandas `merge()` function is at the heart of most data science projects.